In [ ]:


// Waiting a  click to start the BLE device connection process. Once triggered by clcik the following will start.    
document.getElementById('connectButton').addEventListener('click', function() {
    navigator.bluetooth.requestDevice({
        // Use the actual UUID of the BLE service associated with our Arduino device is programmed to broadcast.
        filters: [{ services: ['12ab34cd-56ef-78gh-90ij-123456789abc'] }]
    })
    .then(device => {
        console.log('Connecting to device...');
        return device.gatt.connect();
    })
    .then(server => {
        console.log('Getting Service...');
        // Again, use the same UUID here to retrieve the service
        return server.getPrimaryService('12ab34cd-56ef-78gh-90ij-123456789abc');
    })
    .then(service => {
        console.log('Getting Characteristic...');
        // Here use the UUID of the characteristic that sends gesture data. ? 
        return service.getCharacteristic('characteristic_uuid_here');
    })
    .then(characteristic => {
        console.log('Starting Notifications...');
        characteristic.startNotifications();
        characteristic.addEventListener('characteristicvaluechanged', (event) => {
            const gestureIndex = event.target.value.getUint8(0);
            console.log(`Received gesture index: ${gestureIndex}`);
            document.getElementById('gestureDisplay').textContent = `Gesture Index: ${gestureIndex}`;
            handleGesture(gestureIndex);
        });
    })
    .catch(error => {
        console.error('Connection failed!', error);
    });
});




In [ ]:
// Function to handle the gesture index by making an API call
function handleGesture(gestureIndex) {
    // Define actions for each gesture index
    const actions = {
        0: { // Swipe Up
            enabled: true,
            url: "https://api.example.com/devices/light/on", // Endpoint to turn the light on for Phillips HUE endpoint
            method: "POST"
        },
        1: { // Swipe Down
            enabled: true,
            url: "https://api.example.com/devices/light/off", // Endpoint to turn the light off for Phillips HUE endpoint
            method: "POST"
        }
    };

    // Get the action based on the gesture index
    let action = actions[gestureIndex];

    // Check if the action is defined and enabled
    if (action && action.enabled) {
        // Make the API call using Fetch API
        fetch(action.url, { method: action.method })
        .then(response => response.json()) // Convert the response to JSON
        .then(data => console.log('API response:', data)) // Log the data returned from the API
        .catch(error => console.error('API call failed:', error)); // Log any errors that occur
    } else {
        // Log if no action is configured for the gesture or if it's disabled
        console.log("No action configured for this gesture or action disabled.");
    }
}